In [8]:
# Kaggle - Digit Recognizer 문제 풀기
#경로: https://www.kaggle.com/c/digit-recognizer
#train.csv =>7:3 비율로 나누어 학습 및 accuracy 확인
# - batch형태로 데이터를 읽어들여서 학습

#test.csv 를 이용하여 prediction 결과 도출
#해당 결과를 이용하여 submission.csv파일을 생성한 후 kaggle site 에 제출 및 확인

import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

tf.reset_default_graph()
df = pd.read_csv("./data/recognizer/train.csv",sep=",",skiprows=0)

split_cnt = int(df.shape[0] * 0.7)
train_df = df.loc[:split_cnt,:]
test_df = df.loc[split_cnt:,:]

x_data = MinMaxScaler().fit_transform(train_df.loc[:,"pixel0":])
y_data = tf.one_hot(train_df["label"],10) 
sess = tf.Session()
y_data = sess.run(y_data)

x_data_test = MinMaxScaler().fit_transform(test_df.loc[:,"pixel0":])
y_data_test = tf.one_hot(test_df["label"],10) 
y_data_test = sess.run(y_data_test)

display(x_data.shape) #(42000, 784)
display(y_data.shape) #(42000, 1)

print(type(x_data))
print(type(y_data))
#placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32) 
Y = tf.placeholder(shape=[None,10], dtype=tf.float32)

keep_prob = tf.placeholder(dtype=tf.float32)
#Weight & bias
#행렬곱 연산을 해야하기때문에 [784,10]
#W1 = tf.Variable(tf.random_normal([784,300]), name="weight1") 
W1 = tf.get_variable("weight1", shape=[784,256],initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([256]), name="bias1")
_layer1 = tf.nn.relu(tf.matmul(X,W1) +  b1)
layer1 = tf.nn.dropout(_layer1, keep_prob=keep_prob)

#W2 = tf.Variable(tf.random_normal([300,300]), name="weight2") 
W2 = tf.get_variable("weight2", shape=[256,256],initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]), name="bias2")
_layer2 = tf.nn.relu(tf.matmul(layer1,W2) +  b2)
layer2 = tf.nn.dropout(_layer2, keep_prob=keep_prob)

#W3 = tf.Variable(tf.random_normal([300,10]), name="weight3") 
W3 = tf.get_variable("weight3", shape=[256,10],initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([10]), name="bias3")

#Hypothesis
logits = tf.matmul(layer2,W3) + b3 #행열곱
#H = tf.nn.softmax(logits) #확율값 높은값 추출
H = tf.nn.relu(logits)

#cost function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))

#training node생성
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

#session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습진행
training_epoch = 100 
batch_size = 100 
                 
for step in range(training_epoch):
    num_of_iter = int(x_data.shape[0] / batch_size)
    cost_val = 0
    start_idx = 0
    end_idx = 0
    for i in range(num_of_iter):
        start_idx = (i * training_epoch)
        end_idx = start_idx + training_epoch
        if end_idx > x_data.shape[0]:
            end_idx = x_data.shape[0]
                
        batch_x = x_data[start_idx:end_idx,:]
        batch_y = y_data[start_idx:end_idx,:]
        
        _, cost_val = sess.run([train,cost],feed_dict={X:batch_x,Y:batch_y,keep_prob:1.0})
        
    
    if step % 10 == 0:
        print(cost_val)
        

#Accuracy 정확도 측정
predict = tf.argmax(H,1) #열방향으로 큰값 추출
correct = tf.equal(predict, tf.argmax(Y,1)) #실제값과 예측값 비교
accuracy = tf.reduce_mean(tf.cast(correct,dtype=tf.float32))

result = sess.run(accuracy,feed_dict={X:x_data_test,
                                     Y:y_data_test,keep_prob:1})

print("정확도:{}".format(result))

#predict check
df_test = pd.read_csv("./data/recognizer/test.csv",sep=",",skiprows=0)
test_x_data = MinMaxScaler().fit_transform(df_test)

predict = tf.argmax(H,1)
result = sess.run(predict, feed_dict={X:test_x_data, keep_prob: 1.0})
df = pd.DataFrame({
    'ImageId': [i for i in range(1,28001)],
    'Label': result
})
#display(result)
df.to_csv('./data/recognizer/submission.csv', index=False)


(29400, 784)

(29400, 10)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
0.20917295
0.009655466
0.00070271245
0.00014997795
1.5719643e-05
2.9967466e-06
6.055769e-07
1.1920907e-07
2.0265574e-08
4.7683715e-09
정확도:0.9769859313964844


array([2, 0, 9, ..., 3, 9, 2], dtype=int64)

In [18]:
x_data.shape[0]

29400 // 100

294

In [114]:
# Titanic: Machine Learning from Disaster 문제 풀기
# https://www.kaggle.com/c/titanic/data
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

tf.reset_default_graph()
df = pd.read_csv("./data/titanic/train.csv",sep=",",skiprows=0)

#display(df)

df2 = df[["Survived","Pclass","Sex","Age","Parch","Fare"]]
#df2.dropna(how="any", inplace=True)
df2["Sex"] = df2['Sex'].apply(lambda x: '1' if x == 'male' else '2')
#df2["Age2"] = df2['Age'].apply(lambda x: 0 if (x is None or x == 'NaN') else (x // 10) * 10)
#df2["Age2"] = df2['Age'].apply(lambda x: 0 if (x is None or x == 'NaN') else (x if x < 10 else (x // 10) * 10)
display(df2.loc[:20,:])

split_cnt = int(df2.shape[0] * 0.7)
train_df = df2.loc[:split_cnt,:]
test_df = df2.loc[split_cnt:,:]

df_x = train_df[["Pclass","Sex","Age","Parch","Fare"]]
df_y = train_df[["Survived"]]

x_data = MinMaxScaler().fit_transform(df_x)
y_data = df_y.values

print("x_data type:{}".format(type(x_data)))
print("y_data type:{}".format(type(y_data)))

#placeholder
X = tf.placeholder(shape=[None,5],dtype=tf.float32)
Y = tf.placeholder(shape=[None,1],dtype=tf.float32)
keep_prob = tf.placeholder(dtype=tf.float32)

#Weight & bias
W1 = tf.get_variable("weight1", shape=[5,20],initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([20]), name="bias1")
_layer1 = tf.nn.relu(tf.matmul(X,W1) +  b1)
layer1 = tf.nn.dropout(_layer1, keep_prob=keep_prob)

W2 = tf.get_variable("weight2", shape=[20,20],initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([20]), name="bias2")
_layer2 = tf.nn.relu(tf.matmul(layer1,W2) +  b2)
layer2 = tf.nn.dropout(_layer2, keep_prob=keep_prob)

W3 = tf.get_variable("weight3", shape=[20,1],initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([1]), name="bias3")

#Hypothesis
logits = tf.matmul(layer2,W3) + b3 #행열곱
H = tf.nn.relu(logits)

#cost function
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=Y))

#training node 생성
#train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)
train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

#session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#학습과정 진행
training_epoch = 100 
batch_size = 100 
                 
for step in range(training_epoch):
    num_of_iter = int(x_data.shape[0] / batch_size)
    cost_val = 0
    start_idx = 0
    end_idx = 0
    for i in range(num_of_iter):
        start_idx = (i * training_epoch)
        end_idx = start_idx + training_epoch
        if end_idx > x_data.shape[0]:
            end_idx = x_data.shape[0]
                
        batch_x = x_data[start_idx:end_idx,:]
        batch_y = y_data[start_idx:end_idx,:]
        
        _, cost_val = sess.run([train,cost],feed_dict={X:batch_x,Y:batch_y,keep_prob:1.0})
    
    if step % 10 == 0:
        print(cost_val)

#Accuracy (정확도 측정)
df_x_test = test_df[["Pclass","Sex","Age","Parch","Fare"]]
df_y_test = test_df[["Survived"]]

#Accuracy 정확도 측정
predict = tf.argmax(H,1) #열방향으로 큰값 추출
correct = tf.equal(predict, tf.argmax(Y,1)) #실제값과 예측값 비교
accuracy = tf.reduce_mean(tf.cast(correct,dtype=tf.float32))

x_data_t = MinMaxScaler().fit_transform(df_x_test.values)
y_data_t = df_y_test 

result = sess.run(accuracy,feed_dict={X:x_data_t,
                                     Y:y_data_t,keep_prob:1})

print("정확도:{}".format(result))

#===============================================================
#predict check
df_t = pd.read_csv("./data/titanic/test.csv",sep=",",skiprows=0)

df_t2 = df_t[["PassengerId","Pclass","Sex","Age","Parch","Fare"]]
#df_t2.dropna(how="any", inplace=True)
df_t2["Sex"] = df_t2['Sex'].apply(lambda x: '1' if x == 'male' else '2')

df_ids = df_t2[["PassengerId"]]
df_test = df_t2[["Pclass","Sex","Age","Parch","Fare"]]
#print(df_t2)
test_x_data = MinMaxScaler().fit_transform(df_test.values)
#print(test_x_data)

#predict = tf.argmax(H,1)
predict = tf.cast(H > 0.5, dtype=tf.float32) #예측값
result = sess.run(predict, feed_dict={X:test_x_data, keep_prob: 1.0})
#print(type(result))
print(result)
df = pd.DataFrame({
    'PassengerId': [str(i[0]) for i in df_ids.values],
    'Survived': [int(i[0]) for i in result]
})
print(df)

df.to_csv('./data/titanic/gender_submission.csv', index=False)


C:\ProgramData\Anaconda3\envs\cpu_env\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\envs\cpu_env\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Survived,Pclass,Sex,Age,Parch,Fare,Age2
0,0,3,1,22.0,0,7.2500,20.0
1,1,1,2,38.0,0,71.2833,30.0
2,1,3,2,26.0,0,7.9250,20.0
3,1,1,2,35.0,0,53.1000,30.0
4,0,3,1,35.0,0,8.0500,30.0
5,0,3,1,NaN,0,8.4583,NaN
6,0,1,1,54.0,0,51.8625,50.0
7,0,3,1,2.0,1,21.0750,0.0
8,1,3,2,27.0,2,11.1333,20.0
9,1,2,2,14.0,0,30.0708,10.0


x_data type:<class 'numpy.ndarray'>
y_data type:<class 'numpy.ndarray'>
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
정확도:1.0
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.

C:\ProgramData\Anaconda3\envs\cpu_env\lib\site-packages\ipykernel_launcher.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [97]:
23 // 10

2

In [ ]:
# Titanic: Machine Learning from Disaster 문제 풀기
# https://www.kaggle.com/c/titanic/data
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

tf.reset_default_graph()
df = pd.read_csv("./data/titanic/train.csv",sep=",",skiprows=0)

#display(df)

df2 = df[["Survived","Pclass","Sex","Age","Parch","Fare"]]
#df2.dropna(how="any", inplace=True)
df2["Sex"] = df2['Sex'].apply(lambda x: '1' if x == 'male' else '2')
#df2["Age2"] = df2['Age'].apply(lambda x: 0 if (x is None or x == 'NaN') else (x // 10) * 10)
display(df2.loc[:20,:])

split_cnt = int(df2.shape[0] * 0.7)
train_df = df2.loc[:split_cnt,:]
test_df = df2.loc[split_cnt:,:]

df_x = train_df[["Pclass","Sex","Age","Parch","Fare"]]
df_y = train_df[["Survived"]]

x_data = MinMaxScaler().fit_transform(df_x)
y_data = df_y.values

print("x_data type:{}".format(type(x_data)))
print("y_data type:{}".format(type(y_data)))

#placeholder
X = tf.placeholder(shape=[None,5],dtype=tf.float32)
Y = tf.placeholder(shape=[None,1],dtype=tf.float32)
keep_prob = tf.placeholder(dtype=tf.float32)

#Weight & bias
W1 = tf.get_variable("weight1", shape=[5,20],initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([20]), name="bias1")
_layer1 = tf.nn.relu(tf.matmul(X,W1) +  b1)
layer1 = tf.nn.dropout(_layer1, keep_prob=keep_prob)

W2 = tf.get_variable("weight2", shape=[20,20],initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([20]), name="bias2")
_layer2 = tf.nn.relu(tf.matmul(layer1,W2) +  b2)
layer2 = tf.nn.dropout(_layer2, keep_prob=keep_prob)

W3 = tf.get_variable("weight3", shape=[20,1],initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([1]), name="bias3")

#Hypothesis
logits = tf.matmul(layer2,W3) + b3 #행열곱
H = tf.nn.relu(logits)

#cost function
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=Y))

#training node 생성
#train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)
train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

#session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#학습과정 진행
training_epoch = 100 
batch_size = 100 
                 
for step in range(training_epoch):
    num_of_iter = int(x_data.shape[0] / batch_size)
    cost_val = 0
    start_idx = 0
    end_idx = 0
    for i in range(num_of_iter):
        start_idx = (i * training_epoch)
        end_idx = start_idx + training_epoch
        if end_idx > x_data.shape[0]:
            end_idx = x_data.shape[0]
                
        batch_x = x_data[start_idx:end_idx,:]
        batch_y = y_data[start_idx:end_idx,:]
        
        _, cost_val = sess.run([train,cost],feed_dict={X:batch_x,Y:batch_y,keep_prob:1.0})
    
    if step % 10 == 0:
        print(cost_val)

#Accuracy (정확도 측정)
df_x_test = test_df[["Pclass","Sex","Age","Parch","Fare"]]
df_y_test = test_df[["Survived"]]

#Accuracy 정확도 측정
predict = tf.argmax(H,1) #열방향으로 큰값 추출
correct = tf.equal(predict, tf.argmax(Y,1)) #실제값과 예측값 비교
accuracy = tf.reduce_mean(tf.cast(correct,dtype=tf.float32))

x_data_t = MinMaxScaler().fit_transform(df_x_test.values)
y_data_t = df_y_test 

result = sess.run(accuracy,feed_dict={X:x_data_t,
                                     Y:y_data_t,keep_prob:1})

print("정확도:{}".format(result))

#===============================================================
#predict check
df_t = pd.read_csv("./data/titanic/test.csv",sep=",",skiprows=0)

df_t2 = df_t[["PassengerId","Pclass","Sex","Age","Parch","Fare"]]
#df_t2.dropna(how="any", inplace=True)
df_t2["Sex"] = df_t2['Sex'].apply(lambda x: '1' if x == 'male' else '2')

df_ids = df_t2[["PassengerId"]]
df_test = df_t2[["Pclass","Sex","Age","Parch","Fare"]]
#print(df_t2)
test_x_data = MinMaxScaler().fit_transform(df_test.values)
#print(test_x_data)

#predict = tf.argmax(H,1)
predict = tf.cast(H > 0.5, dtype=tf.float32) #예측값
result = sess.run(predict, feed_dict={X:test_x_data, keep_prob: 1.0})
#print(type(result))
print(result)
df = pd.DataFrame({
    'PassengerId': [str(i[0]) for i in df_ids.values],
    'Survived': [int(i[0]) for i in result]
})
print(df)

df.to_csv('./data/titanic/gender_submission.csv', index=False)
